In [1]:
# Dependencies
import json
import gmaps
import pandas as pd
import requests
from pprint import pprint
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [2]:
city_df = pd.read_csv("Output_csv_files/city_data.csv")
city_df.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,ribeira grande,38.52,-28.70,61.54,73.0,97.0,4.70
1,punta arenas,-53.15,-70.92,41.00,80.0,20.0,13.87
2,wattegama,6.80,81.48,84.13,62.0,1.0,4.97
3,ushuaia,-54.80,-68.30,41.00,81.0,20.0,5.82
4,hobart,-42.88,147.33,53.69,58.0,75.0,20.80


In [3]:
locations = city_df[["Latitude","Longitude"]]
humid = city_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(51.5, -0.2), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
vacation_df = pd.DataFrame(city_df, columns = ["City", "Temperature", "Wind Speed", "Cloudiness"])

temp_param = (vacation_df["Temperature"] <= 80) & (vacation_df["Temperature"] > 70)
wind_param = vacation_df["Wind Speed"] < 10
cloud_param = vacation_df["Cloudiness"] == 0
temp_param
vacation_df[temp_param & wind_param & cloud_param]
#len(vacation_df[temp_param & wind_param & cloud_param])

,City,Temperature,Wind Speed,Cloudiness
20,lazaro cardenas,74.05,1.63,0.0
91,dwarka,78.91,9.55,0.0
113,morondava,76.64,4.41,0.0
148,birin,71.55,4.09,0.0
231,pisco,71.76,8.05,0.0
308,bageshwar,75.25,0.54,0.0
437,miandrivazo,75.99,3.36,0.0
474,antsohihy,75.96,8.84,0.0
494,mehnagar,75.20,2.24,0.0


In [6]:
final_vacation_df = vacation_df[temp_param & wind_param & cloud_param]
final_vacation_df

,City,Temperature,Wind Speed,Cloudiness
20,lazaro cardenas,74.05,1.63,0.0
91,dwarka,78.91,9.55,0.0
113,morondava,76.64,4.41,0.0
148,birin,71.55,4.09,0.0
231,pisco,71.76,8.05,0.0
308,bageshwar,75.25,0.54,0.0
437,miandrivazo,75.99,3.36,0.0
474,antsohihy,75.96,8.84,0.0
494,mehnagar,75.20,2.24,0.0


In [7]:
hotel_df = pd.DataFrame(city_df, columns = ["City", "Latitude", "Longitude"])
hotel_df["Hotel Name"] = ""
hotel_df

,City,Latitude,Longitude,Hotel Name
0,ribeira grande,38.52,-28.70,
1,punta arenas,-53.15,-70.92,
2,wattegama,6.80,81.48,
3,ushuaia,-54.80,-68.30,
4,hobart,-42.88,147.33,
...,...,...,...,...
569,mehamn,71.04,27.85,
570,mogadishu,2.04,45.34,
571,alice springs,-23.70,133.88,
572,hegang,47.40,130.37,


In [8]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": gkey
}
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"]= name_address["results"][0]["name"]
        print(name_address["results"][0]["name"])
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Quinta Das Buganvílias
Patagonia B&B
Missing field/result... skipping.
Los Cauquenes Resort & Spa
Hotel Grand Chancellor Hobart
Pebbles Inn
Hôtel Restaurant Sakouli
Old Colony Inn
Bertelli Chuí Hotel
Missing field/result... skipping.
Missing field/result... skipping.
Ellie's Villa
Playa Linda Hotel
Hotel Meuligoe
Oa Oa Lodge
Missing field/result... skipping.
Broadwater Resort
Paradise First Hotel
Missing field/result... skipping.
Splendid Inn Bayshore
Hotel de Casablanca
Nusa Island Retreat
Esperance Seafront Holiday Units
Hotel Madira
Missing field/result... skipping.
Atlanticview Cape Town Boutique Hotel
Best Western Thompson Hotel & Suites
Victoria Hotel - Bredasdorp
Missing field/result... skipping.
Pousada Pura Vida Club Brasil
Pension Maro'i
Hotel Plaza Lebu
Gostinitsa Ip Zabirov
Missing field/result... skipping.
Ahipara Bay Motel
Sofitel Legend Old Cataract Aswan
Mar Guesthouse
DISCOVERY AMBOASARY
Missing field/result... skipping.
THE REDWOOD RIVERWALK HOTEL
Powell River Town Ce

Best Western Burke and Wills Motor Inn
诚信商务酒店
Gacayte Hotel
Missing field/result... skipping.
Hotel Carabane
Shellharbour Resort & Conference Centre
Hilton Garden Inn Norwalk
Missing field/result... skipping.
Missing field/result... skipping.
The Celtic Manor Resort
Holy Drop Skate House
Holiday Inn Express Campbellsville
Missing field/result... skipping.
Shangri-La Hotel, The Marina, Cairns
Missing field/result... skipping.
Every day Holiday
Hotelli Puustelli
Opuwo Country Lodge
Shapur Hotel
Missing field/result... skipping.
Hotel Maniitsoq
Missing field/result... skipping.
Madina white house hotel
Missing field/result... skipping.
Sea View Hotel
Missing field/result... skipping.
Quality Hotel Mildura Grand
Три Сосны Гостинница
Azalea Luxury Lodge
Hilton Garden Inn Lincoln Downtown/Haymarket
AungKaDay Hotel
Hotel Grot
Bari Hotel
Missing field/result... skipping.
Mövenpick Hotel Qassim
Meieriet Restaurant
Yongchang Hotel
Gostinitsa Zverevo
Missing field/result... skipping.
Inn Gostinny

In [11]:
#Save dataframe as csv
hotel_df.to_csv("Output_csv_files/hotel_output.csv", encoding="utf-8", index=False)
hotel_df.head(20)

,City,Latitude,Longitude,Hotel Name
0,ribeira grande,38.52,-28.70,Quinta Das Buganvílias
1,punta arenas,-53.15,-70.92,Patagonia B&B
2,wattegama,6.80,81.48,
3,ushuaia,-54.80,-68.30,Los Cauquenes Resort & Spa
4,hobart,-42.88,147.33,Hotel Grand Chancellor Hobart
5,hithadhoo,-0.60,73.08,Pebbles Inn
6,bandrele,-12.91,45.19,Hôtel Restaurant Sakouli
7,new norfolk,-42.78,147.06,Old Colony Inn
8,chuy,-33.70,-53.46,Bertelli Chuí Hotel
9,lavrentiya,65.58,-171.00,
